In [82]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import numpy as np
from hmmlearn import hmm
import random
import warnings
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
warnings.filterwarnings('ignore')

In [83]:
data_path = 'HI-Small_Trans_processed_w_original.csv' # after categorical encoder, and field selection
df = pd.read_csv(data_path)
print(df.shape)
df = df.drop_duplicates()
print(df.shape)
df.head()

(5078345, 18)
(5078336, 18)


,Timestamp,FromAccount,ToAccount,FromBank,ToBank,ReceivingCurrency,PaymentCurrency,PaymentFormat,AmountPaid,AmountReceived,FromBankOriginal,ToBankOriginal,FromAccountOriginal,ToAccountOriginal,ReceivingCurrencyOriginal,PaymentCurrencyOriginal,PaymentFormatOriginal,IsLaundering
0,2022-09-01 00:20:00,6530,6530,8,8,12,12,5,3697.34,3697.34,10,10,8000EBD30,8000EBD30,US Dollar,US Dollar,Reinvestment,0
1,2022-09-01 00:20:00,358174,176809,109,0,12,12,3,0.01,0.01,3208,1,8000F4580,8000F5340,US Dollar,US Dollar,Cheque,0
2,2022-09-01 00:00:00,358476,358476,110,110,12,12,5,14675.57,14675.57,3209,3209,8000F4670,8000F4670,US Dollar,US Dollar,Reinvestment,0
3,2022-09-01 00:02:00,74640,74640,10,10,12,12,5,2806.97,2806.97,12,12,8000F5030,8000F5030,US Dollar,US Dollar,Reinvestment,0
4,2022-09-01 00:06:00,6538,6538,8,8,12,12,5,36682.97,36682.97,10,10,8000F5200,8000F5200,US Dollar,US Dollar,Reinvestment,0


In [84]:
def group_by_accounts(df, col1, col2, other_columns):
    # Reshape the DataFrame to have a single account column, while preserving other specified columns
    melted_df = pd.melt(df.reset_index(), id_vars=['index'] + other_columns, value_vars=[col1, col2])
    melted_df.rename(columns={'value': 'Account'}, inplace=True)
    
    # Group by account and collect all unique indices for each account
    account_indices = melted_df.groupby('Account')['index'].unique()
    
    # Create a DataFrame for each account group using the collected indices
    account_group_dataframes = {account: df.loc[indices].drop_duplicates() for account, indices in account_indices.items()}
    return account_group_dataframes

In [85]:
%%time

other_columns = [x for x in df.columns if x not in ['FromAccount', 'ToAccount']]
grouped_data = group_by_accounts(df, 'FromAccount', 'ToAccount', other_columns)

CPU times: total: 17min 36s
Wall time: 17min 59s


In [86]:
print(len(grouped_data))

515088


In [87]:
for k, v in grouped_data.items():
    print(k)
    print(v)
    break

0
                  Timestamp  FromAccount  ToAccount  FromBank  ToBank  \
212996  2022-09-01 00:22:00       320763          0      6940     598   

        ReceivingCurrency  PaymentCurrency  PaymentFormat  AmountPaid  \
212996                 13               13              3        0.01   

        AmountReceived  FromBankOriginal  ToBankOriginal FromAccountOriginal  \
212996            0.01            310070           10057           803B2E780   

       ToAccountOriginal ReceivingCurrencyOriginal PaymentCurrencyOriginal  \
212996         801FB1090                       Yen                     Yen   

       PaymentFormatOriginal  IsLaundering  
212996                Cheque             0  


In [88]:
min_len = 35
test_ratio = 0.2

import tqdm
X_train, X_test = [], []
for this_name, this_seq in tqdm.tqdm(grouped_data.items()):
    if this_seq.shape[0] < min_len:
        X_train.append(this_seq)
        continue
    test_len = int(this_seq.shape[0] * test_ratio)
    train_len = this_seq.shape[0] - test_len
    X_train.append(this_seq[:train_len])
    X_test.append(this_seq[train_len:])

print(len(X_train))
print(len(X_test))


100%|██████████| 515088/515088 [00:06<00:00, 77801.53it/s] 

515088
92971


In [89]:
train_data_seq = pd.concat(X_train).drop_duplicates()
test_data_seq = pd.concat(X_test).drop_duplicates()

In [90]:
print(train_data_seq.shape)
print(test_data_seq.shape)

(4948983, 18)
(1069155, 18)


In [91]:
test_data = df.loc[test_data_seq.index]
test_data.shape

(1069155, 18)

In [92]:
test_data.head()

,Timestamp,FromAccount,ToAccount,FromBank,ToBank,ReceivingCurrency,PaymentCurrency,PaymentFormat,AmountPaid,AmountReceived,FromBankOriginal,ToBankOriginal,FromAccountOriginal,ToAccountOriginal,ReceivingCurrencyOriginal,PaymentCurrencyOriginal,PaymentFormatOriginal,IsLaundering
2798149,2022-09-06 00:45:00,281020,2,19,598,13,13,3,1391409.78,1391409.78,24,10057,803A6D150,803AA8E90,Yen,Yen,Cheque,0
3023096,2022-09-06 12:22:00,152746,2,81,598,13,13,3,1983640.26,1983640.26,1686,10057,800F2FE90,803AA8E90,Yen,Yen,Cheque,0
3023098,2022-09-06 12:09:00,152746,2,81,598,13,13,4,1481597.58,1481597.58,1686,10057,800F2FE90,803AA8E90,Yen,Yen,Credit Card,0
3719466,2022-09-07 23:21:00,152746,2,81,598,13,13,3,1983640.26,1983640.26,1686,10057,800F2FE90,803AA8E90,Yen,Yen,Cheque,0
3719468,2022-09-07 23:05:00,152746,2,81,598,13,13,4,1481597.58,1481597.58,1686,10057,800F2FE90,803AA8E90,Yen,Yen,Credit Card,0


In [94]:
# From Bank	Account	To Bank	Account.1	Amount Received	Receiving Currency	Amount Paid	Payment Currency	Payment Format
# FromAccountOriginal	ToAccountOriginal	ReceivingCurrencyOriginal	PaymentCurrencyOriginal	PaymentFormatOriginal
test_data.rename(columns={
    'FromBankOriginal': 'From Bank', 
    'ToBankOriginal': 'To Bank', 
    'FromAccountOriginal': 'Account', 
    'ToAccountOriginal': 'Account.1',
    'ReceivingCurrencyOriginal': 'Receiving Currency',
    'PaymentCurrencyOriginal': 'Payment Currency',
    'PaymentFormatOriginal': 'Payment Format',
    'IsLaundering': 'Is Laundering'
    }, inplace=True)
test_data.head()

,Timestamp,FromAccount,ToAccount,FromBank,ToBank,ReceivingCurrency,PaymentCurrency,PaymentFormat,AmountPaid,AmountReceived,From Bank,To Bank,Account,Account.1,Receiving Currency,Payment Currency,Payment Format,Is Laundering
2798149,2022-09-06 00:45:00,281020,2,19,598,13,13,3,1391409.78,1391409.78,24,10057,803A6D150,803AA8E90,Yen,Yen,Cheque,0
3023096,2022-09-06 12:22:00,152746,2,81,598,13,13,3,1983640.26,1983640.26,1686,10057,800F2FE90,803AA8E90,Yen,Yen,Cheque,0
3023098,2022-09-06 12:09:00,152746,2,81,598,13,13,4,1481597.58,1481597.58,1686,10057,800F2FE90,803AA8E90,Yen,Yen,Credit Card,0
3719466,2022-09-07 23:21:00,152746,2,81,598,13,13,3,1983640.26,1983640.26,1686,10057,800F2FE90,803AA8E90,Yen,Yen,Cheque,0
3719468,2022-09-07 23:05:00,152746,2,81,598,13,13,4,1481597.58,1481597.58,1686,10057,800F2FE90,803AA8E90,Yen,Yen,Credit Card,0


In [95]:
train_data = df.loc[~df.index.isin(test_data_seq.index)]
train_data.shape

(4009181, 18)

In [96]:
train_data.head()

,Timestamp,FromAccount,ToAccount,FromBank,ToBank,ReceivingCurrency,PaymentCurrency,PaymentFormat,AmountPaid,AmountReceived,FromBankOriginal,ToBankOriginal,FromAccountOriginal,ToAccountOriginal,ReceivingCurrencyOriginal,PaymentCurrencyOriginal,PaymentFormatOriginal,IsLaundering
0,2022-09-01 00:20:00,6530,6530,8,8,12,12,5,3697.34,3697.34,10,10,8000EBD30,8000EBD30,US Dollar,US Dollar,Reinvestment,0
1,2022-09-01 00:20:00,358174,176809,109,0,12,12,3,0.01,0.01,3208,1,8000F4580,8000F5340,US Dollar,US Dollar,Cheque,0
2,2022-09-01 00:00:00,358476,358476,110,110,12,12,5,14675.57,14675.57,3209,3209,8000F4670,8000F4670,US Dollar,US Dollar,Reinvestment,0
3,2022-09-01 00:02:00,74640,74640,10,10,12,12,5,2806.97,2806.97,12,12,8000F5030,8000F5030,US Dollar,US Dollar,Reinvestment,0
4,2022-09-01 00:06:00,6538,6538,8,8,12,12,5,36682.97,36682.97,10,10,8000F5200,8000F5200,US Dollar,US Dollar,Reinvestment,0


In [97]:
train_data.rename(columns={
    'FromBankOriginal': 'From Bank', 
    'ToBankOriginal': 'To Bank', 
    'FromAccountOriginal': 'Account', 
    'ToAccountOriginal': 'Account.1',
    'ReceivingCurrencyOriginal': 'Receiving Currency',
    'PaymentCurrencyOriginal': 'Payment Currency',
    'PaymentFormatOriginal': 'Payment Format',
    'IsLaundering': 'Is Laundering'
    }, inplace=True)
train_data.head()

,Timestamp,FromAccount,ToAccount,FromBank,ToBank,ReceivingCurrency,PaymentCurrency,PaymentFormat,AmountPaid,AmountReceived,From Bank,To Bank,Account,Account.1,Receiving Currency,Payment Currency,Payment Format,Is Laundering
0,2022-09-01 00:20:00,6530,6530,8,8,12,12,5,3697.34,3697.34,10,10,8000EBD30,8000EBD30,US Dollar,US Dollar,Reinvestment,0
1,2022-09-01 00:20:00,358174,176809,109,0,12,12,3,0.01,0.01,3208,1,8000F4580,8000F5340,US Dollar,US Dollar,Cheque,0
2,2022-09-01 00:00:00,358476,358476,110,110,12,12,5,14675.57,14675.57,3209,3209,8000F4670,8000F4670,US Dollar,US Dollar,Reinvestment,0
3,2022-09-01 00:02:00,74640,74640,10,10,12,12,5,2806.97,2806.97,12,12,8000F5030,8000F5030,US Dollar,US Dollar,Reinvestment,0
4,2022-09-01 00:06:00,6538,6538,8,8,12,12,5,36682.97,36682.97,10,10,8000F5200,8000F5200,US Dollar,US Dollar,Reinvestment,0


In [98]:
train_data['Is Laundering'].value_counts()

Is Laundering
0    4005109
1       4072
Name: count, dtype: int64

In [99]:
test_data['Is Laundering'].value_counts()

Is Laundering
0    1068050
1       1105
Name: count, dtype: int64

In [100]:
test_data.to_csv(f'HI-Small_Trans_processed_w_original_test.csv', index=False)
train_data.to_csv(f'HI-Small_Trans_processed_w_original_train.csv', index=False)